# things to do:
# -decide on module type# things to do:
# -decide on module type



In [ ]:
#Installing Missing Content
!pip install torch
!pip install lightning

# Importing Data se# Importing Dataset

# Formatting Data for Learning# Formatting Data for Learning

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import lightning as pl
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch import Trainer
from torchmetrics import Accuracy


blackjack_raw = pd.read_csv('./blkjckhands.csv')
blackjack_raw

,Unnamed: 0,PlayerNo,card1,card2,card3,card4,card5,sumofcards,dealcard1,dealcard2,...,dealcard4,dealcard5,sumofdeal,blkjck,winloss,plybustbeat,dlbustbeat,plwinamt,dlwinamt,ply2cardsum
0,0,Player1,7,10,0,0,0,17,10,8,...,0,0,18,nowin,Loss,Beat,Dlwin,0,10,17
1,1,Player2,10,9,0,0,0,19,10,8,...,0,0,18,nowin,Win,Plwin,Beat,20,0,19
2,2,Player3,9,8,0,0,0,17,10,8,...,0,0,18,nowin,Loss,Beat,Dlwin,0,10,17
3,3,Player4,2,10,0,5,0,17,10,8,...,0,0,18,nowin,Loss,Beat,Dlwin,0,10,12
4,4,Player5,10,2,0,5,0,17,10,8,...,0,0,18,nowin,Loss,Beat,Dlwin,0,10,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,1,Player2,10,7,0,0,0,17,3,9,...,2,0,18,nowin,Loss,Beat,Dlwin,0,10,17
899996,2,Player3,6,1,10,0,0,17,3,9,...,2,0,18,nowin,Loss,Beat,Dlwin,0,10,7
899997,3,Player4,4,2,9,0,0,15,3,9,...,2,0,18,nowin,Loss,Beat,Dlwin,0,10,6
899998,4,Player5,9,10,0,0,0,19,3,9,...,2,0,18,nowin,Win,Plwin,Beat,20,0,19


In [ ]:
did_hit = blackjack_raw['card3'] != 0
blackjack_raw['didhit'] = did_hit
blackjack_raw

,Unnamed: 0,PlayerNo,card1,card2,card3,card4,card5,sumofcards,dealcard1,dealcard2,...,dealcard5,sumofdeal,blkjck,winloss,plybustbeat,dlbustbeat,plwinamt,dlwinamt,ply2cardsum,didhit
0,0,Player1,7,10,0,0,0,17,10,8,...,0,18,nowin,Loss,Beat,Dlwin,0,10,17,False
1,1,Player2,10,9,0,0,0,19,10,8,...,0,18,nowin,Win,Plwin,Beat,20,0,19,False
2,2,Player3,9,8,0,0,0,17,10,8,...,0,18,nowin,Loss,Beat,Dlwin,0,10,17,False
3,3,Player4,2,10,0,5,0,17,10,8,...,0,18,nowin,Loss,Beat,Dlwin,0,10,12,False
4,4,Player5,10,2,0,5,0,17,10,8,...,0,18,nowin,Loss,Beat,Dlwin,0,10,12,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,1,Player2,10,7,0,0,0,17,3,9,...,0,18,nowin,Loss,Beat,Dlwin,0,10,17,False
899996,2,Player3,6,1,10,0,0,17,3,9,...,0,18,nowin,Loss,Beat,Dlwin,0,10,7,True
899997,3,Player4,4,2,9,0,0,15,3,9,...,0,18,nowin,Loss,Beat,Dlwin,0,10,6,True
899998,4,Player5,9,10,0,0,0,19,3,9,...,0,18,nowin,Win,Plwin,Beat,20,0,19,False


In [ ]:
filtered_data = blackjack_raw[['card1', 'card2', 'didhit', 'dealcard1', 'winloss']]
filtered_data

,card1,card2,didhit,dealcard1,winloss
0,7,10,False,10,Loss
1,10,9,False,10,Win
2,9,8,False,10,Loss
3,2,10,False,10,Loss
4,10,2,False,10,Loss
...,...,...,...,...,...
899995,10,7,False,3,Loss
899996,6,1,True,3,Loss
899997,4,2,True,3,Loss
899998,9,10,False,3,Win


In [ ]:
isWin = filtered_data['winloss'] != 'Loss'
filtered_data['winloss'] = isWin
filtered_data.head(3)

<ipython-input-5-2d75f7c25504>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['winloss'] = isWin


,card1,card2,didhit,dealcard1,winloss
0,7,10,False,10,False
1,10,9,False,10,True
2,9,8,False,10,False


In [ ]:
#hyper params:
input_size = 4
hidden_size = 12
num_classes = 2
num_epochs = 10
batch_size = 12
lr = 0.001
#Convert to usable dataset:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
        features = torch.tensor(self.data.iloc[idx, :4], dtype=torch.float32)
        return features, target



dataset = CustomDataset(filtered_data)
(train_dataset, val_dataset) = torch.utils.data.random_split(dataset, (0.7, 0.3))

In [ ]:
#NEURAL NET CLASS:::::
class ligNeuralNet(pl.LightningModule):
    def __init__(self,input_size,hid_size,num_classes):
        super(ligNeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hid_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hid_size, num_classes)
        self.accuracy = Accuracy(task='multiclass', num_classes = num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)

        return out
    #+===============================================================================================================+
    #TRAINING FUNCTIONS:
    def training_step(self, batch, batch_idx):
        features, target = batch
        #print(type(batch))
        # Forward pass
        outputs = self(features)
        loss = F.cross_entropy(outputs, target)

        self.accuracy(outputs, target)
        self.log('acc', self.accuracy, prog_bar=True)
        self.log('loss', loss)
        return loss

    # define what happens for testing here

    def train_dataloader(self):
        # Data loader
        return torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
    #+===============================================================================================================+
    #VALIDATION FUNCTIONS:
    def val_dataloader(self):
        return torch.utils.data.DataLoader(dataset = val_dataset, batch_size=batch_size, num_workers=4, shuffle=False)

    def validation_step(self, batch, batch_idx):
        inputs, target = batch
        outputs = self.forward(inputs)
        self.accuracy(outputs, target)
        self.log('val_acc', self.accuracy, prog_bar=True)

    #+===============================================================================================================+

    #+===============================================================================================================+
    #test functions:
    def predict_dataloader(self):##############
        return torch.utils.data.DataLoader(dataset = test_dataset, batch_size=batch_size, num_workers=4, shuffle=False)

    def predict_step(self, batch, batch_idx):
        features, target = batch
        y_pred= self(features)
        #for x in y_pred:
        #    print(torch.argmax(x))
        return y_pred.softmax(dim=-1)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=lr)

In [ ]:
logger = CSVLogger('./lightning_logs/', 'lstm')
trainer = Trainer(max_epochs = num_epochs, logger = logger)
model = ligNeuralNet(input_size, hidden_size, num_classes)
trainer.fit(model)

"""
test_dataset = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=batch_size, num_workers=4, shuffle=False)
trainer.predict(model, test_dataset, return_predictions=False)#figure out how to pass this function a dataloader. logger = CSVLogger('./lightning_logs/', 'lstm')
trainer = Trainer(max_epochs = num_epochs, fast_dev_run = True, logger = logger)
model = ligNeuralNet(input_size, hidden_size, num_classes)
trainer.fit(model)

trainer.predict(model, return_predictions=False)#figure out how to pass this function a dataloader.
"""


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name     | Type               | Params
------------------------------------------------
0 | l1       | Linear             | 60    
1 | relu     | ReLU               | 0     
2 | l2       | Linear             | 26    
3 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
86        Trainable params
0         Non-trainable params
86        Total params
0.000     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be 

Training: |          | 0/? [00:00<?, ?it/s]

<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)


...

Validation: |          | 0/? [00:00<?, ?it/s]

<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[idx, 4], dtype=torch.long)
<ipython-input-6-f0175aad10a5>:17: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  target = torch.tensor(self.data.iloc[id

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


"\ntest_dataset = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=batch_size, num_workers=4, shuffle=False)\ntrainer.predict(model, test_dataset, return_predictions=False)#figure out how to pass this function a dataloader. logger = CSVLogger('./lightning_logs/', 'lstm')\ntrainer = Trainer(max_epochs = num_epochs, fast_dev_run = True, logger = logger)\nmodel = ligNeuralNet(input_size, hidden_size, num_classes)\ntrainer.fit(model)\n\ntrainer.predict(model, return_predictions=False)#figure out how to pass this function a dataloader. \n"

In [ ]:
#
class PredictionDataSet(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        target = torch.tensor([1])
        features = torch.tensor(self.data.iloc[idx, :4], dtype=torch.float32)
        return features, target

In [ ]:
card1 = float(input("what is your first card"))
card2 = float(input("what is your second card"))
dealercard = float(input("what is the dealers card"))

prediction = [[card1, card2, 1.0, dealercard], [card1, card2, 0.0, dealercard]]

inputFrame = pd.DataFrame(prediction)
inputFrame = PredictionDataSet(inputFrame)
inputFrame = torch.utils.data.DataLoader(dataset=inputFrame, batch_size=2, num_workers=1, shuffle=False)

#trainer.predict(model, inputFrame, return_predictions=False)
#model.eval()
#model(torch.tensor([card1, card2, 1, dealercard]))
data = trainer.predict(model, inputFrame, return_predictions=True)
if (data[0][0][0].item() > data[0][1][0].item()):
    print("HIT")
else:
    print("STAND")
data

what is your first card3
what is your second card4
what is the dealers card3


Predicting: |          | 0/? [00:00<?, ?it/s]

HIT


[tensor([[0.5127, 0.4873],
         [0.3261, 0.6739]])]